# Get gender

## Add `'gender'` key to database

In [2]:
import requests
from IPython.display import JSON
from gender_guesser_first_names import gender
from re import sub
from re import search
from re import UNICODE
import json
import country_converter as coco

In [3]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [4]:
import pandas as pd

In [6]:
DF = pd.DataFrame(df)
DFA = DF[~DF['author_id'].isna()]
DF.shape,DFA.shape

((57588, 13), (57576, 13))

In [7]:
DFB = DF[DF['author_id'].isna()].reset_index(drop=True)
DFB['author_id']=[f'unknown{str(x).zfill(2)}' for x in range(DFB.shape[0])]
DFB['author_id'] = DFB['author_id'].replace('unknown05','unknown04')

In [8]:
DF = pd.concat((DFA,DFB)).reset_index(drop=True)
del DFA
del DFB
DF.shape

(57588, 13)

In [9]:
def remove_initials(d):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    if (d.get('name') and d.get('name').get('value') ):
        first_names = d.get('name').get('value').split(', ')[-1].replace('-',' ').strip()
    elif d.get('full_name'):
        first_names = d.get('full_name').split(', ')[-1].replace('-',' ').strip()
    else:
        first_names = ''
    x = first_names
    x = sub(r'[\|†•]','',x).strip()
    x = sub(r'\.',' ',   x, UNICODE).strip()
    x = sub(r'\s\w\s','',x, UNICODE).strip()
    x = sub(r'^\w\s','',x, UNICODE).strip()
    x = sub(r'\s\w$','',x, UNICODE).strip()
    x = sub(r'^\w$','',x, UNICODE).strip()
    x = sub(r'^[A-Z]{2}$','',x, UNICODE)
    
    return x

In [10]:
DF['names'] = DF.apply(lambda d: remove_initials(d), axis='columns')

In [11]:
#DF = DF[DF['names'].apply(len)>0]
DF.shape

(57588, 14)

In [12]:
DF[DF['names'].apply(len)==0][:3]

,author_id,country,email_addresses,full_name,ids,institution,institution_id,name,papers,positions,profile_id,advisors,gender,names
12,J.Ospino.1,Spain,None,"Ospino, J.","[{'value': 'J.Ospino.1', 'schema': 'INSPIRE BA...","Salamanca U., IUFFyM",910114,"{'value': 'Ospino, J.', 'preferred_name': 'J. ...","[{'recid': 1781327, 'year': '2020', 'citation_...",[{'record': {'$ref': 'https://inspirehep.net/a...,1041300,[],unknown,
24,J.H.Pena.1,Venezuela,None,"Pena, J.M.","[{'value': 'J.H.Pena.1', 'schema': 'INSPIRE BA...","Caracas, U. Central",902716,"{'value': 'Pena, J.M.', 'preferred_name': 'J.M...","[{'recid': 781916, 'year': '2008', 'citation_c...",[{'record': {'$ref': 'https://inspirehep.net/a...,1052800,[],unknown,
29,A.J.Kalnay.1,Venezuela,None,"Kalnay, A.J.",None,Simon Bolivar U.,903205,None,"[{'recid': 93012, 'year': '1974', 'citation_co...",None,2204420,[],unknown,


In [13]:
DF[DF['names'].str.split().apply(lambda L: 1 in [len(x) for x in L])].names

Series([], Name: names, dtype: object)

In [14]:
DF[DF['names'].str.split().apply(lambda L: 2 in [len(x) for x in L])].names.unique()[:3]

array(['Le', 'Luis Ye', 'Kee Su'], dtype=object)

In [15]:
DFU = DF[['country']].drop_duplicates()

In [16]:
DFU['country_id'] = DFU['country'].astype(str).replace('Serbia and Montenegro','Serbia').apply(lambda c: coco.convert(names=c, to='ISO2')).replace('not found','')

None not found in regex


In [17]:
DF = DF.merge(DFU,on='country',how='left')

In [18]:
del DFU

In [19]:
DF['country_id'].unique()

array(['VE', 'CL', 'RW', 'ZA', 'US', 'ES', 'FR', 'CH', 'IT', 'DE', 'PE',
       'AR', 'UY', 'BR', 'EC', 'GB', 'CO', 'IL', 'BG', 'MX', 'NL', 'JP',
       'CA', 'SE', 'IR', 'IN', 'SI', 'BY', 'AU', 'RU', 'BE', 'HU', 'DK',
       '', 'SK', 'TH', 'CZ', 'JO', 'GR', 'CU', 'KW', 'FI', 'UA', 'AT',
       'TR', 'RS', 'PT', 'AM', 'KR', 'CN', 'MT', 'PL', 'EE', 'RO', 'HR',
       'DO', 'GT', 'IS', 'PY', 'PA', 'HN', 'PH', 'SV', 'PK', 'OM', 'CR',
       'NZ', 'GE', 'BO', 'KZ', 'VN', 'AE', 'NO', 'EG', 'TW', 'PR', 'JM',
       'LT', 'ID', 'LU', 'DZ', 'ET', 'SA', 'IE', 'CY', 'VA', 'MD', 'MK',
       'LB', 'BD', 'QA', 'MA', 'SG', 'MN', 'UZ', 'BS', 'BJ', 'LK', 'IQ',
       'AZ', 'SY', 'NG', 'PS', 'GH', 'KG', 'CG', 'ME', 'NE', 'BN', 'LV',
       'MY', 'MG', 'BT', 'TN', 'UG', 'BA'], dtype=object)

In [64]:
DFU = DF[['author_id','names','country','country_id']].rename({'names':'name'},axis='columns').drop_duplicates('author_id').reset_index(drop=True)
DFU.shape

(29979, 4)

In [65]:
D=gender.Detector(case_sensitive=False)
def force_get_gender(d,first_names='names'):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    gender = 'unknown'
    names = d.get(first_names)
    if names:
        if d.get('country'):
            gender = D.get_gender( names, country=d.get('country').strip() )
        else:
            gender = D.get_gender( names )
    return gender

In [66]:
DFU['gender'] = DFU.apply(lambda d: force_get_gender(d,first_names='name'), axis='columns')

In [67]:
DFU.shape

(29979, 5)

In [68]:
DFUA = DFU[DFU['name'].apply(len)>0].reset_index(drop=True)
DFUB = DFU[DFU['name'].apply(len)==0].reset_index(drop=True)
DFU.shape[0], DFUA.shape[0], DFUB.shape[0] + DFUA.shape[0],round( DFUB.shape[0] / DFU.shape[0],2)

(29979, 20125, 29979, 0.33)

In [71]:
DFUA.shape

(20125, 5)

In [72]:
DFUA.to_csv('/home/restrepo/Downloads/genderize.csv',index=False)

In [73]:
csv = 'https://genderize.io/download/csv/eyJjbGFzc2lmaWNhdGlvbl9maWxlbmFtZSI6Ijc4N2Q5ZmYwLTVhOGQtNDkxNy1hYmMwLWZhZjhlZGUyZDQ1MSIsInVwbG9hZF9maWxlbmFtZSI6IjNlMjA3Zjg1LWFjMWItNDdhZC05Y2MyLWI2MGUyZjJlMjI3MSIsInVwbG9hZF9vcmlnaW5hbF9uYW1lIjoiZ2VuZGVyaXplLmNzdiJ9'
kk = pd.read_csv(csv)

In [75]:
ls -lh /home/restrepo/Downloads/genderize.csv

-rw-rw-r-- 1 restrepo restrepo 785K nov 15 11:02 /home/restrepo/Downloads/genderize.csv


In [70]:
kk

,author_id,name,country,country_id,gender,Gender,Gender Probability,Gender Count
0,A.M.Kotzinian.1,Aram,Armenia,AM,male,male,1.0,1313
1,N.Sahakyan.1,Narek,Armenia,AM,male,male,1.0,1151


In [ ]:
f=open('data/inspire_LA.json','w')
json.dump(df,f)
f.close()

## Analysis

In [ ]:
import pandas as pd

In [ ]:
DF = pd.read_json('data/inspire_LA.json')

Unique authors ids

In [ ]:
DF.shape[0]

In [ ]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape[0]

In [ ]:
DF['first_names'] = DF['name'].str.get('value').astype(str).str.split(', ').str[-1]#.apply(
    #lambda s: sub('\w\.','',s)).str.replace('None','')

In [ ]:
DF['first_names'] = DF.apply(lambda row: str(row['full_name']).split(', ')[-1]  
                             if row['first_names']=='None' 
                             else row['first_names'] ,axis = 'columns')

In [ ]:
DF['gender'].unique()

In [ ]:
print(DF['gender'].shape[0])
total = DF['gender'].dropna().shape[0]
total

In [ ]:
male = DF[DF['gender'] == 'male'].shape[0]
female = DF[DF['gender'] == 'female'].shape[0]
unknown = DF[DF['gender'] == 'unknown'].shape[0]
unknown = unknown + DF[DF['gender'] == 'andy'].shape[0]

In [ ]:
assert male + female + unknown == total

In [ ]:
male+female

In [ ]:
male

In [ ]:
female

In [ ]:
round(female/male,2)

In [ ]:
unknown

In [ ]:
DFFM = DF[ (( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFFM.shape

In [ ]:
DFU = DF[ ~(( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFU.shape

In [ ]:
DFFM[['author_id','first_names','country','gender']].sample(50)

In [ ]:
#[['first_names','country','gender']]
DFU.sample(50)[['author_id','full_name','name','country','gender']]

check specific entries

In [ ]:
JSON([d for d in df if d.get('author_id')=='F.Fontanot.3'][0])

In [ ]:
import requests
au = requests.get('https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1')
if au.status_code == 200:
    au = au.json()

In [ ]:
len(au['hits']['hits'])

In [ ]:
au['links']['next']

In [ ]:
# https://github.com/inspirehep/rest-api-doc/issues/20#issuecomment-997932079
url='https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1&q=control_number:[1+TO+2852058]'
au = requests.get(url)
if au.status_code == 200:
    au = au.json()

In [ ]:
len(au['hits']['hits'])

In [ ]:
au.status_code

In [ ]:
au = requests.get(url)

In [ ]:
step = 10000
ini = 981872
end = ini + step
for i in range(187):
    print(f'control_number:[{ini} TO {end}]')
    ini = end + 1
    end = ini + step

In [ ]:
au.status_code

In [ ]:
len(au.json()['hits']['hits'])